In [ ]:
import numpy as np
import rioxarray as rxr
import matplotlib.pyplot as plt
from IPython.display import clear_output
from osgeo import gdal_array

In [ ]:
%%time
# Read raster data as numeric array from file
image_filename = '/att/gpfsfs/briskfs01/ppl/mwooten3/Vietnam_LCLUC/TOA/M1BS/pansharpen_TamNongClip/WV02_20130125_M1BS_103001001DD53500-toa_pansharpen_clip.tif'
rasterArray = gdal_array.LoadFile(image_filename)
rasterArray.shape

In [ ]:
type(rasterArray)

In [ ]:
%%time
image_filename = '/att/gpfsfs/briskfs01/ppl/mwooten3/Vietnam_LCLUC/TOA/M1BS/pansharpen_TamNongClip/WV02_20130125_M1BS_103001001DD53500-toa_pansharpen_clip.tif'
image_data = rxr.open_rasterio(image_filename)
image_array = image_data.values
image_array.shape

In [ ]:
import scipy.signal
import scipy.signal.windows as w
'''
Generates a 2D square image (of size window_size) containing a 2D user-defined
window with values ranging from 0 to 1.
It is possible to pass arguments to the window function by setting kwargs.
All available windows: https://docs.scipy.org/doc/scipy/reference/signal.windows.html
'''
def window2d(window_func, window_size, **kwargs):
    window = np.matrix(window_func(M=window_size, sym=False, **kwargs))
    return window.T.dot(window)

fig, axes = plt.subplots(1, 3, figsize=(15, 6), sharey=True)
for i, ax in enumerate(axes.ravel()):
    ax.set_xlabel(['(a)', '(b)', '(c)'][i], fontsize=20)
    ax.set_xticks([])
    ax.set_yticks([])
    if i == 3:
        pyra = get_loss_weight(128, 128, 1, stride_height=0, stride_width=0, batch_size=1, border =0)
        ax.imshow(pyra, cmap='plasma')
    else:
        w_type = [w.hann, w.barthann, w.triang][i]
        im = ax.imshow(window2d(w_type, 128), cmap='plasma', vmin=0, vmax=1)
plt.colorbar(im, ax=axes.flat, pad=0.04, fraction=0.0105)
plt.show()

In [ ]:
def generate_corner_windows(window_func, window_size, **kwargs):
    step = window_size >> 1
    window = window2d(window_func, window_size, **kwargs)
    window_u = np.vstack([np.tile(window[step:step+1, :], (step, 1)), window[step:, :]])
    window_b = np.vstack([window[:step, :], np.tile(window[step:step+1, :], (step, 1))])
    window_l = np.hstack([np.tile(window[:, step:step+1], (1, step)), window[:, step:]])
    window_r = np.hstack([window[:, :step], np.tile(window[:, step:step+1], (1, step))])
    window_ul = np.block([
        [np.ones((step, step)), window_u[:step, step:]],
        [window_l[step:, :step], window_l[step:, step:]]])
    window_ur = np.block([
        [window_u[:step, :step], np.ones((step, step))],
        [window_r[step:, :step], window_r[step:, step:]]])
    window_bl = np.block([
        [window_l[:step, :step], window_l[:step, step:]],
        [np.ones((step, step)), window_b[step:, step:]]])
    window_br = np.block([
        [window_r[:step, :step], window_r[:step, step:]],
        [window_b[step:, :step], np.ones((step, step))]])
    return np.array([
        [ window_ul, window_u, window_ur ],
        [ window_l,  window,   window_r  ],
        [ window_bl, window_b, window_br ],
    ])

In [ ]:
def generate_patch_list(image_width, image_height, window_func, window_size, overlapping=False):
    patch_list = []
    if overlapping:
        step = window_size >> 1
        windows = generate_corner_windows(window_func, window_size)
        max_height = int(image_height/step - 1)*step
        max_width = int(image_width/step - 1)*step
    else:
        step = window_size
        windows = np.ones((window_size, window_size))
        max_height = int(image_height/step)*step
        max_width = int(image_width/step)*step
    for i in range(0, max_height, step):
        for j in range(0, max_width, step):
            if overlapping:
                # Close to border and corner cases
                # Default (1, 1) is regular center window
                border_x, border_y = 1, 1
                if i == 0: border_x = 0
                if j == 0: border_y = 0
                if i == max_height-step: border_x = 2
                if j == max_width-step: border_y = 2
                # Selecting the right window
                current_window = windows[border_x, border_y]
            else:
                current_window = windows
            # The patch is cropped when the patch size is not
            # a multiple of the image size.
            patch_height = window_size
            if i+patch_height > image_height:
                patch_height = image_height - i
            patch_width = window_size
            if j+patch_width > image_width:
                patch_width = image_width - j
            # Adding the patch
            patch_list.append(
                (j, i, patch_width, patch_height, current_window[:patch_height, :patch_width])
            )
    return patch_list

In [ ]:
%%time
"""
[(0,
  0,
  256,
  256,
  array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.35477166e-03, 6.02271897e-04, 1.50590652e-04],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.35477166e-03, 6.02271897e-04, 1.50590652e-04],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.35477166e-03, 6.02271897e-04, 1.50590652e-04],
         ...,
         [1.35477166e-03, 1.35477166e-03, 1.35477166e-03, ...,
          1.83540625e-06, 8.15940899e-07, 2.04015948e-07],
         [6.02271897e-04, 6.02271897e-04, 6.02271897e-04, ...,
          8.15940899e-07, 3.62731438e-07, 9.06965177e-08],
         [1.50590652e-04, 1.50590652e-04, 1.50590652e-04, ...,
          2.04015948e-07, 9.06965177e-08, 2.26775444e-08]])),
"""
window_func = w.hann
use_hanning = True
image_data_shape = [1024, 1024] #image_data.shape
tile_size = 128
M = np.zeros((image_data_shape[0], image_data_shape[1])) # Output image
patch_list = generate_patch_list(image_data_shape[0], image_data_shape[1], window_func, tile_size, use_hanning)
len(patch_list)

In [ ]:
%%time
counter = 0
for patch in patch_list:
    counter += 1
    print(patch)
    #patch_x, patch_y, patch_width, patch_height, window = patch
    #input_path = image_data[patch_y:patch_y+patch_height, patch_x:patch_x+patch_width].values
    
    # prediction = np.ones_like(window) * np.random.randint(0, 2) # Simulated patch
    # input_path = input_image[patch_y:patch_y+patch_height, patch_x:patch_x+patch_width]
    # prediction = model(input_patch)
    #input_path = image_data[patch_y:patch_y+patch_height, patch_x:patch_x+patch_width].data
    #print(input_path.data)
    #prediction = model(input_patch)
    #M[patch_y:patch_y+patch_height, patch_x:patch_x+patch_width] += prediction * window
print(counter)

In [ ]:
def generate_patch_list(image_width, image_height, window_func, window_size, overlapping=False):
    patch_list = []
    if overlapping:
        step = window_size >> 1
        windows = generate_corner_windows(window_func, window_size)
        max_height = int(image_height/step - 1)*step
        max_width = int(image_width/step - 1)*step
    else:
        step = window_size
        windows = np.ones((window_size, window_size))
        max_height = int(image_height/step)*step
        max_width = int(image_width/step)*step
    for i in range(0, max_height, step):
        for j in range(0, max_width, step):
            if overlapping:
                # Close to border and corner cases
                # Default (1, 1) is regular center window
                border_x, border_y = 1, 1
                if i == 0: border_x = 0
                if j == 0: border_y = 0
                if i == max_height-step: border_x = 2
                if j == max_width-step: border_y = 2
                # Selecting the right window
                current_window = windows[border_x, border_y]
            else:
                current_window = windows
            # The patch is cropped when the patch size is not
            # a multiple of the image size.
            patch_height = window_size
            if i+patch_height > image_height:
                patch_height = image_height - i
            patch_width = window_size
            if j+patch_width > image_width:
                patch_width = image_width - j
            # Adding the patch
            patch_list.append(
                (j, i, patch_width, patch_height, current_window[:patch_height, :patch_width])
            )
    return patch_list

In [ ]:
tile_size = 8192
image_shape = (38702, 71223, 4)
step = tile_size >> 1

max_height = int(image_shape[0]/step)*step
max_width = int(image_shape[1]/step)*step
print(max_height, max_width)

In [ ]:
counter = 0
for i in range(0, max_height, step):
    for j in range(0, max_width, step):
        print(i, j)
        counter += 1
print(counter)